In [10]:
# from future import print_function
import os
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import torch.utils.data as data
import sys
sys.path.append(os.path.abspath(".."))
from data import WiderFaceDetection, detection_collate, preproc, cfg_mnet, cfg_re50
from layers.modules import MultiBoxLoss
from layers.functions.prior_box import PriorBox
import time
import datetime
import math
from models.retinaface import RetinaFace
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2

In [ ]:
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

print(f"Using device: {device}")

In [ ]:
# Declare config (use resnet50 backbone)
cfg = cfg_re50
rgb_mean = (104, 117, 123) # bgr order
num_classes = 2
img_dim = cfg['image_size']
num_gpu = cfg['ngpu']
batch_size = cfg['batch_size']
max_epoch = cfg['epoch']
gpu_train = cfg['gpu_train']

initial_lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [ ]:
csv_file = "../data/fairface/fairface_label_train.csv"
df = pd.read_csv(csv_file)

# Separate male and female images
female_images = df[df["gender"] == 'Female']['file'].tolist()
male_images = df[df['gender'] == 'Male']['file'].tolist()

print(f"Number of Female Images: {len(female_images)}")
print(f"Number of Male Images: {len(male_images)}")

Number of Female Images: 40758
Number of Male Images: 45986


In [ ]:
# Instantiate model with the configurations above
model = RetinaFace(cfg=cfg).to(device)
model.eval()

In [ ]:
def detect_faces(image_path):
    image = cv2.imread(image_path)
    detections = model.detect_faces(image)

    if detections:
        return len(detections), [d['score'] for d in detections]
    return 0, []